In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps

plt.rcParams['font.size'] = 11


# Dataset “Babyboom”

- Проверьте гипотезу, если средний вес девочек такой же, как вес мальчиков (t-тест).
- Проверьте гипотезу, если дисперсия веса девочек такая же, как и веса мальчиков (F-тест или Левена).

In [ ]:
babyboomDataFrame = pd.read_csv('../datasets/babyboom.dat.txt', sep='\s+', header=None,
                                names=['timeOfBirth24', 'sexCode', 'birthWeightGrams', 'minutesAfterMidnight'])

babyboomDataFrame['sexCode'] = babyboomDataFrame['sexCode'].astype(int)
babyboomDataFrame['birthWeightGrams'] = pd.to_numeric(babyboomDataFrame['birthWeightGrams'], errors='coerce')

print('Размер выборки (всего):', len(babyboomDataFrame))
display(babyboomDataFrame.head())


In [ ]:
weightSeries = babyboomDataFrame['birthWeightGrams'].dropna()
girlsSeries = babyboomDataFrame.loc[babyboomDataFrame['sexCode']==1, 'birthWeightGrams'].dropna()
boysSeries = babyboomDataFrame.loc[babyboomDataFrame['sexCode']==2, 'birthWeightGrams'].dropna()

def printSummary(series, label):
    print(f"\n--- {label} ---")
    print('N =', series.size, 'Mean =', series.mean(), 'SD =', series.std(ddof=1))

    shStat, shP = sps.shapiro(series)
    print('Shapiro-Wilk statistic =', round(shStat,6), ', p-value =', round(shP,6))

    muHat = series.mean()
    sigmaHat = series.std(ddof=1)
    ksStat, ksP = sps.kstest(series, 'norm', args=(muHat, sigmaHat))
    print('Kolmogorov-Smirnov statistic =', round(ksStat,6), ', p-value =', round(ksP,6))

printSummary(weightSeries, 'All')
printSummary(girlsSeries, 'Girls')
printSummary(boysSeries, 'Boys')


In [ ]:
# Гипотеза о равенстве средних (t-тест)
tStatMeans, tPvalueMeans = sps.ttest_ind(girlsSeries, boysSeries, equal_var=False)
print('\n=== Тест на равенство средних (Welch\'s t-test) ===')
print('t-statistic =', round(tStatMeans, 4), ', p-value =', round(tPvalueMeans, 6))

# Гипотеза о равенстве дисперсий (Levene тест)
leveneStat, levenePvalue = sps.levene(girlsSeries, boysSeries)
print('\n=== Тест на равенство дисперсий (Levene) ===')
print('Levene statistic =', round(leveneStat, 4), ', p-value =', round(levenePvalue, 6))

# Альтернатива: F-тест для дисперсий
fStat = np.var(girlsSeries, ddof=1) / np.var(boysSeries, ddof=1)
fPvalue = 2 * min(sps.f.cdf(fStat, len(girlsSeries)-1, len(boysSeries)-1), 
                  sps.f.sf(fStat, len(girlsSeries)-1, len(boysSeries)-1))
print('F-statistic =', round(fStat, 4), ', p-value =', round(fPvalue, 6))


# Dataset “Euroweight”

- Проверить гипотезы о том, что среднее значение веса монеты одинаково в разных пакетах (попарно и все вместе).

In [ ]:
euroweightDataFrame = pd.read_csv('../datasets/euroweight.dat.txt', sep='\s+', header=None,
                                  names=['index','weight','batch'])
euroweightDataFrame['weight'] = pd.to_numeric(euroweightDataFrame['weight'], errors='coerce')
euroweightDataFrame['batch'] = euroweightDataFrame['batch'].astype(int)

display(euroweightDataFrame.head())
print('Размер выборки:', len(euroweightDataFrame))


In [ ]:
# ANOVA для всех батчей (равенство средних)
batchGroups = [group['weight'].dropna().values for name, group in euroweightDataFrame.groupby('batch')]
anovaF, anovaP = sps.f_oneway(*batchGroups)
print('=== ANOVA для равенства средних по всем батчам ===')
print('F-statistic =', round(anovaF, 4), ', p-value =', round(anovaP, 6))

# Попарные t-тесты (Welch's)
print('\n=== Попарные t-тесты (Welch\'s) ===')
batches = sorted(euroweightDataFrame['batch'].unique())
for i in range(len(batches)):
    for j in range(i+1, len(batches)):
        batch1 = euroweightDataFrame.loc[euroweightDataFrame['batch']==batches[i], 'weight'].dropna().values
        batch2 = euroweightDataFrame.loc[euroweightDataFrame['batch']==batches[j], 'weight'].dropna().values
        tStat, tP = sps.ttest_ind(batch1, batch2, equal_var=False)
        print(f'Batch {batches[i]} vs {batches[j]}: t = {round(tStat,4)}, p = {round(tP,6)}')


# Dataset “Iris”

- Проверить гипотезы о равенстве распределений характеристик цветков разных типов (KS-тест попарно).
- Проверьте гипотезы о равенстве средних (ANOVA) и дисперсий (Levene) различных характеристик цветов разных типов.

In [ ]:
irisDataFrame = pd.read_csv('../datasets/iris.txt', header=None,
                            names=['sepalLength','sepalWidth','petalLength','petalWidth','class'])
irisDataFrame['sepalLength'] = pd.to_numeric(irisDataFrame['sepalLength'], errors='coerce')
irisDataFrame['sepalWidth'] = pd.to_numeric(irisDataFrame['sepalWidth'], errors='coerce')
irisDataFrame['petalLength'] = pd.to_numeric(irisDataFrame['petalLength'], errors='coerce')
irisDataFrame['petalWidth'] = pd.to_numeric(irisDataFrame['petalWidth'], errors='coerce')

print('Размер выборки:', len(irisDataFrame))
print('Классы:', irisDataFrame['class'].unique())
display(irisDataFrame.head())


In [ ]:
features = ['sepalLength', 'sepalWidth', 'petalLength', 'petalWidth']
classes = sorted(irisDataFrame['class'].unique())

for feature in features:
    print(f'\n=== {feature.upper()} ===')
    
    # ANOVA для средних
    groups = [irisDataFrame.loc[irisDataFrame['class']==cls, feature].dropna().values for cls in classes]
    anovaF, anovaP = sps.f_oneway(*groups)
    print('ANOVA (средние): F =', round(anovaF, 4), ', p =', round(anovaP, 6))
    
    # Levene для дисперсий
    leveneStat, leveneP = sps.levene(*groups)
    print('Levene (дисперсии): stat =', round(leveneStat, 4), ', p =', round(leveneP, 6))


In [ ]:
# Попарные KS-тесты для распределений
for feature in features:
    print(f'\n=== KS-тесты для {feature.upper()} (попарно) ===')
    for i in range(len(classes)):
        for j in range(i+1, len(classes)):
            group1 = irisDataFrame.loc[irisDataFrame['class']==classes[i], feature].dropna().values
            group2 = irisDataFrame.loc[irisDataFrame['class']==classes[j], feature].dropna().values
            ksStat, ksP = sps.ks_2samp(group1, group2)
            print(f'{classes[i]} vs {classes[j]}: D = {round(ksStat,4)}, p = {round(ksP,6)}')


# Dataset “Surgery”

- Проверить гипотезу об успешности операции с вероятностью 0.7 (0.8). Под успехом мы подразумеваем, что «V справа» до операции меньше, чем «V справа» после операции, и одновременно с этим «V слева» до операции меньше, чем «V слева» после операции.

In [ ]:
surgeryDataFrame = pd.read_excel('../datasets/surgery.xlsx', header=1)

display(surgeryDataFrame.head())

surgeryDataFrame.rename(columns={
    surgeryDataFrame.columns[0]: 'beforeRight',
    surgeryDataFrame.columns[1]: 'beforeLeft',
    surgeryDataFrame.columns[2]: 'afterRight',
    surgeryDataFrame.columns[3]: 'afterLeft'
}, inplace=True)

def toFloatSafe(x):
    try:
        if pd.isna(x):
            return np.nan
        return float(str(x).replace(',', '.'))
    except:
        return np.nan

for col in ['beforeRight','beforeLeft','afterRight','afterLeft']:
    surgeryDataFrame[col + 'Val'] = surgeryDataFrame[col].apply(toFloatSafe)

surgeryClean = surgeryDataFrame.dropna(subset=['beforeRightVal','beforeLeftVal','afterRightVal','afterLeftVal']).copy()

surgeryClean['success'] = (
    (surgeryClean['afterRightVal'] > surgeryClean['beforeRightVal']) &
    (surgeryClean['afterLeftVal'] > surgeryClean['beforeLeftVal'])
)

numPatients = surgeryClean.shape[0]
numSuccesses = int(surgeryClean['success'].sum())

print('Количество пациентов с полными данными =', numPatients)
print('Количество успехов =', numSuccesses)


In [ ]:
# Биномиальный тест для p=0.7
binomTest07 = sps.binomtest(numSuccesses, numPatients, p=0.7, alternative='two-sided')
print('=== Биномиальный тест для p=0.7 ===')
print('statistic =', binomTest07.statistic, ', p-value =', round(binomTest07.pvalue, 6))

# Биномиальный тест для p=0.8
binomTest08 = sps.binomtest(numSuccesses, numPatients, p=0.8, alternative='two-sided')
print('\n=== Биномиальный тест для p=0.8 ===')
print('statistic =', binomTest08.statistic, ', p-value =', round(binomTest08.pvalue, 6))
